In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("E:/data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143634 entries, 0 to 143633
Data columns (total 5 columns):
Unnamed: 0      143634 non-null int64
date            143634 non-null object
sales_amount    143634 non-null float64
item_code       143634 non-null object
category        143634 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 5.5+ MB


In [2]:
#Extracting relevant columns
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data[['date', 'sales_amount', 'category', 'item_code']]
data['category'] = data['category'].astype('category')
data['item_code'] = data['item_code'].astype('category')
category = data[['category','item_code']]

# Item-wise Weekly Sales

In [3]:
#Grouping data by weekly sales for each item code
group = data.groupby([pd.Grouper(key='date',freq='W-MON'),'item_code']).agg({'sales_amount': ['sum']})
group.columns = ['sales']
group = group.reset_index()
group['sales'].fillna(0, inplace=True)

In [4]:
#Extracting category of each item code
join = pd.merge(group,category,on='item_code',how='left')
join['week_start']=join['date']- pd.to_timedelta(7, unit='d') #Calculating week end date
df = join[['week_start','category','item_code','sales']] #Extracting relavnt columns
df=df.drop_duplicates(keep='first')

In [5]:
#Extracting entries for one category
df = df[df['category'].str.match('category_name')]

# Threshold as Coefficient of Variance

In [6]:
#Calculating standard deviation for each item code based on weekly sales figures
stdev = df.groupby(['item_code']).agg({'sales': ['std']})
stdev.columns = ['stdev']
stdev = stdev.reset_index()
thold = pd.merge(df,stdev,on='item_code',how='left')
thold = thold.drop_duplicates(keep='first')

In [7]:
#Calculating mean for each item code based on weekly sales figures
mean = df.groupby(['item_code']).agg({'sales': ['mean']})
mean.columns = ['mean']
mean = mean.reset_index()
threshold = pd.merge(thold,mean,on='item_code',how='left')
threshold=threshold.drop_duplicates(keep='first')

In [ ]:
#Calculating coefficient of variation
threshold['coeff_var']=threshold['stdev']/threshold['mean']

#Extracting only those entries with coefficient of variation less than 4.5 and positive
threshold = threshold[(threshold['coeff_var'] >0) & (threshold['coeff_var'] <4.5)]
threshold.reset_index(drop=True)